# Init

In [1]:
%load_ext autoreload

In [2]:
!pipenv install "langchain==0.0.335"

Installing langchain==0.0.335...
Resolving langchain==0.0.335...
✔ Installation Succeeded
⠋ Installing langchain...
Installing dependencies from Pipfile.lock (b2f8d6)...
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.


In [3]:
!pipenv install --deploy --system

Installing dependencies from Pipfile.lock (b2f8d6)...


In [4]:
import langchain
langchain.__version__

'0.0.335'

In [190]:
from develop.utils import formatters

In [191]:
from llmlib.app import app
context = app.app_context()
context.push()

In [192]:
from langchain.agents import initialize_agent, AgentType, Tool
from langchain.chains import LLMMathChain
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.utilities import SerpAPIWrapper, SQLDatabase

In [193]:
from llmlib.database import Neo4j

In [194]:
graph = Neo4j().graph(name='neo4j')
graph.schema = ''  # Overwrite schema to avoid it to be loaded from db (lengthy process not ussed in here)

#### Init graph search API wrapper

In [195]:
from llmlib.utils.search.cypher_search_api_wrapper import CypherSearchAPIWrapper

In [196]:
search = CypherSearchAPIWrapper(
    graph=graph,
    verbose=True,
    return_intermediate_steps=True
)
search

CypherSearchAPIWrapper(graph=<llmlib.utils.lazy_neo4j_graph.LazyNeo4jGraph object at 0x7ffa8cd55a10>, related_nodes_query='\nUNWIND $terms AS term\nMATCH (s:Synonym {lowercase_name: toLower(term)})<-[:HAS_SYNONYM]-(n)\nWHERE n.eid IS NOT NULL AND NOT n.eid IN $exclude_eids\nWITH apoc.text.levenshteinSimilarity(term, n.name) as similarity, s, n, term\nORDER BY similarity DESC\nWHERE similarity > 0.5\nWITH DISTINCT term, collect({node: n, node_id: id(n), similarity: similarity})[0..$top_k_matches] as matches\nRETURN term, matches\n', related_nodes_query_params={'top_k_matches': 5, 'exclude_eids': ['NAD-P-OR-NOP', 'NADH-P-OR-NOP', 'Donor-H2', 'Acceptor', 'HYDROGEN-PEROXIDE', 'OXYGEN-MOLECULE', 'NAD', 'NADP', 'NADH', 'NADPH', 'WATER', 'CARBON-DIOXIDE', 'FAD', 'CO-A', 'UDP', 'AMMONIA', 'NA+', 'AMMONIUM', 'PROTON', 'CARBON-MONOXIDE', 'GTP', 'ADP', 'GDP', 'AMP', 'ATP', '3-5-ADP', 'PPI', 'Pi']}, relationships_query="\nUNWIND apoc.coll.combinations($term_match, 2) AS termPair\nUNWIND termPair[0]['matches'] as matchA\nUNWIND termPair[1]['matches'] as matchB\nWITH matchA, matchB, matchA['similarity'] * matchB['similarity'] AS similarity\nORDER BY similarity DESC\nWHERE similarity > 0.5\nMATCH (a), (b)\nWHERE id(a) = matchA['id'] AND id(b) = matchB['id']\nMATCH path=allShortestPaths((a)-[*1..]-(b))\nWHERE\n all(r in relationships(path) where type(r) in $include_rels)\n and all(n in nodes(path) where not n.eid in $exclude_eids)\nRETURN path LIMIT $top_k\n", relationships_query_params={'top_k': 3, 'include_rels': ['CONSUMED_BY', 'PRODUCES', 'CATALYZES', 'REGULATES', 'HAS_TXONOMY', 'IS', 'HAS_KO', 'ELEMENT_OF', 'ENCODES'], 'exclude_eids': ['NAD-P-OR-NOP', 'NADH-P-OR-NOP', 'Donor-H2', 'Acceptor', 'HYDROGEN-PEROXIDE', 'OXYGEN-MOLECULE', 'NAD', 'NADP', 'NADH', 'NADPH', 'WATER', 'CARBON-DIOXIDE', 'FAD', 'CO-A', 'UDP', 'AMMONIA', 'NA+', 'AMMONIUM', 'PROTON', 'CARBON-MONOXIDE', 'GTP', 'ADP', 'GDP', 'AMP', 'ATP', '3-5-ADP', 'PPI', 'Pi']})

In [197]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
def tap(func):
    def wrap(arg):
        print(f"tap:{arg}")
        return func(arg)
    return wrap
    
tools = [
    Tool(
        name="Relationship_search",
        func=lambda _: "Zn2+ produces glycolate",
        description="Usefull to get information about relationship between two nodes ids"
    ),
    Tool(
        name="Node_finder",
        func=tap(search.get_related_nodes),
        description="Usefull to determine node id and properties"
    )
]

In [198]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

In [199]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [200]:
from langchain.tools.render import format_tool_to_openai_function

In [201]:
llm_with_tools = llm.bind(functions=[format_tool_to_openai_function(t) for t in tools])

In [202]:
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [203]:
from langchain.agents import AgentExecutor

In [204]:
agent_executor = initialize_agent(
    agent=AgentType.OPENAI_MULTI_FUNCTIONS,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=len(tools) + 1
)

In [205]:
# Do this so we can see exactly what's going on under the hood
from langchain.globals import set_debug

set_debug(True)

In [206]:
agent_executor.invoke(
    {
        "input": "What is the relationship between term zn2+ and glycolate? What properties does zn2+ have?"
    }
)

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "What is the relationship between term zn2+ and glycolate? What properties does zn2+ have?"
}
[llm/start] [1:chain:AgentExecutor > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: You are a helpful AI assistant.\nHuman: What is the relationship between term zn2+ and glycolate? What properties does zn2+ have?"
  ]
}
[llm/end] [1:chain:AgentExecutor > 2:llm:ChatOpenAI] [3.45s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "",
        "generation_info": {
          "finish_reason": "function_call"
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "",
            "additional_kwargs": {
              "function_call

{'input': 'What is the relationship between term zn2+ and glycolate? What properties does zn2+ have?',
 'output': 'The relationship between Zn2+ and glycolate is not found. \n\nZn2+ is the ion of zinc with a charge of +2. It is a transition metal ion and has several properties, including:\n\n1. Atomic number: 30\n2. Atomic mass: 65.38 g/mol\n3. Electron configuration: [Ar] 3d10 4s2\n4. Oxidation state: +2\n5. Physical state: Solid\n6. Color: Bluish-white\n7. Melting point: 419.53°C\n8. Boiling point: 907°C\n9. Density: 7.14 g/cm3\n10. Solubility: Soluble in water and acids\n\nZn2+ is commonly found in various biological systems and plays important roles in enzyme catalysis, DNA synthesis, and immune function.'}